In [ ]:

# libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import style
from pybaseball import statcast
import progressbar 
import warnings
import psycopg2

# pd.options.mode.chained_assignment = None  # default='warn'
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.options.display.float_format = '{:.3f}'.format

df_sc = statcast(start_dt='2024-01-01', end_dt='2024-09-30')


# creating an spray degree column
df_sc['spray_deg'] = np.degrees(np.arctan((df_sc['hc_x'] - 125.42) /  (200 - df_sc['hc_y'])* 0.75 ))

# creating a spray_deg adjusted that accounts for LHH 
df_sc['spray_deg_adj'] = np.where(df_sc['stand'] == 'L', df_sc['spray_deg'] * -1, df_sc['spray_deg'])

df_sc['outcome'] = np.where(df_sc['events'].isin([
    'field_out', 'force_out', 'grounded_into_double_play', 'sac_fly', 'field_error',
    'sac_bunt', 'fielders_choice', 'double_play', 'fielders_choice_out', 
    'sac_fly_double_play', 'triple_play'
]), 'out', df_sc['events'])

# df_sc['zone'] = pd.to_numeric(df_sc['zone'], errors='coerce').astype(int)


This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 200/200 [02:25<00:00,  1.38it/s]
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


ValueError: cannot convert NA to integer

In [ ]:

## outcome, zone, exit_velocity

df_sc




,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,spray_deg,spray_deg_adj,outcome
199,FF,2024-09-30,97.4,-2.1,4.88,"Díaz, Edwin",518595,621242,field_out,hit_into_play,...,1,3,1,1.4,0.96,0.96,17.6,-10.135516,-10.135516,out
204,SL,2024-09-30,90.7,-2.14,5.06,"Díaz, Edwin",518595,621242,None,ball,...,1,3,1,2.14,-0.2,-0.2,23.1,<NA>,<NA>,None
210,SL,2024-09-30,91.1,-2.07,5.14,"Díaz, Edwin",518595,621242,None,swinging_strike,...,1,3,1,2.37,-0.12,-0.12,22.4,<NA>,<NA>,None
214,SL,2024-09-30,91.3,-2.05,5.07,"Díaz, Edwin",518595,621242,None,ball,...,1,3,1,2.09,-0.21,-0.21,22.4,<NA>,<NA>,None
221,SL,2024-09-30,89.1,-2.13,5.15,"Díaz, Edwin",518595,621242,None,swinging_strike,...,1,3,1,2.2,-0.17,-0.17,20.2,<NA>,<NA>,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733,None,2024-03-15,<NA>,<NA>,<NA>,"Singer, Brady",666310,663903,None,swinging_strike,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None
1754,None,2024-03-15,<NA>,<NA>,<NA>,"Singer, Brady",666310,663903,None,swinging_strike,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None
1839,None,2024-03-15,<NA>,<NA>,<NA>,"Singer, Brady",608070,663903,double,hit_into_play,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,36.008672,-36.008672,double
2002,None,2024-03-15,<NA>,<NA>,<NA>,"Singer, Brady",665926,663903,field_out,hit_into_play,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,24.696744,-24.696744,out


In [7]:
import pandas as pd
import numpy as np
from scipy.stats import skewnorm
from tqdm import tqdm

df = df_sc.copy()

# Define EV bucket
def get_ev_bucket(v):
    if pd.isna(v):
        return None
    return f"{int(v // 5) * 5}-{int(v // 5) * 5 + 5}"

df['ev_bucket'] = df['launch_speed'].apply(get_ev_bucket)

# Only keep rows with launch_angle and outcome
df = df[df['launch_angle'].notna() & df['outcome'].notna()]

fallback_levels = [
    ['outcome', 'zone', 'ev_bucket'],
    ['outcome', 'zone'],
    ['outcome'],
    []
]

angle_fits = {}
completed_keys = set()

outer_group_cols = ['batter', 'game_year', 'stand', 'p_throws']
outer_grouped = df.groupby(outer_group_cols)

for outer_key, outer_df in tqdm(outer_grouped, desc="Top-level groups"):
    outer_key = (outer_key,) if not isinstance(outer_key, tuple) else outer_key

    for level_keys in fallback_levels:
        if level_keys:
            grouped = outer_df.groupby(level_keys)
            for key, group in grouped:
                key = (key,) if not isinstance(key, tuple) else key
                full_key_id = (outer_key, tuple(level_keys), key)

                if full_key_id in completed_keys:
                    continue

                angles = pd.to_numeric(group['launch_angle'], errors='coerce').dropna()
                angles = angles[angles.apply(lambda x: isinstance(x, (int, float)) and np.isfinite(x))]

                if len(angles) < 10:
                    continue

                try:
                    a, loc, scale = skewnorm.fit(angles.to_numpy(dtype=np.float64))
                    angle_fits[full_key_id] = {
                        'batter': outer_key[0],
                        'game_year': outer_key[1],
                        'stand': outer_key[2],
                        'p_throws': outer_key[3],
                        'keys': level_keys,
                        'key_values': key,
                        'skew': a,
                        'mean': loc,
                        'std': scale,
                        'n': len(angles)
                    }
                    completed_keys.add(full_key_id)
                except Exception as e:
                    print(f"Skipping {key} at level {level_keys} for group {outer_key} due to error: {e}")
        else:
            # Global fallback
            angles = pd.to_numeric(outer_df['launch_angle'], errors='coerce').dropna()
            angles = angles[angles.apply(lambda x: isinstance(x, (int, float)) and np.isfinite(x))]

            if len(angles) >= 10:
                try:
                    a, loc, scale = skewnorm.fit(angles.to_numpy(dtype=np.float64))
                    angle_fits[(outer_key, (), ())] = {
                        'batter': outer_key[0],
                        'game_year': outer_key[1],
                        'stand': outer_key[2],
                        'p_throws': outer_key[3],
                        'keys': [],
                        'key_values': (),
                        'skew': a,
                        'mean': loc,
                        'std': scale,
                        'n': len(angles)
                    }
                except Exception as e:
                    print(f"Skipping global fallback for {outer_key} due to error: {e}")


Top-level groups: 100%|██████████| 1731/1731 [02:15<00:00, 12.79it/s]


In [15]:
flattened = []

for _, fit in angle_fits.items():
    row = {
        'batter': fit['batter'],
        'game_year': fit['game_year'],
        'stand': fit['stand'],
        'p_throws': fit['p_throws'],
        'skew': fit['skew'],
        'mean': fit['mean'],
        'std': fit['std'],
        'n': fit['n'],
        'level': "_".join(fit['keys']) if fit['keys'] else 'global'
    }

    context_keys = ['outcome', 'zone', 'ev_bucket']
    context_map = dict(zip(fit['keys'], fit['key_values']))

    for key in context_keys:
        val = context_map.get(key, pd.NA)
        if key == 'zone':
            try:
                val = int(val) if pd.notna(val) and float(val).is_integer() else pd.NA
            except:
                val = pd.NA
        row[key] = val

    flattened.append(row)

# Build DataFrame
la_dist_df = pd.DataFrame(flattened)

# Set column order
expected_columns = [
    'game_year', 'batter', 'stand', 'p_throws',
    'outcome', 'zone', 'ev_bucket',
    'skew', 'mean', 'std', 'n', 'level'
]
la_dist_df = la_dist_df[expected_columns]

# Clean up
la_dist_df['zone'] = pd.to_numeric(la_dist_df['zone'], errors='coerce').astype('Int64')
object_cols = la_dist_df.select_dtypes(include='object').columns
la_dist_df[object_cols] = la_dist_df[object_cols].fillna('')

# Export CSV
la_dist_df.to_csv(
    "/Users/loganmottley/Desktop/Projects/go-baseball/data/la_distributions_by_batter_year_handedness.csv",
    index=False,
    na_rep=''
)


In [14]:
la_dist_df



,game_year,batter,stand,p_throws,outcome,zone,ev_bucket,skew,mean,std,n,level
0,2024,444482,L,L,,<NA>,,-8.713282e-01,19.652126,27.943713,12,global
1,2024,444482,L,R,out,4,,4.237722e+00,-26.476535,40.131905,20,outcome_zone
2,2024,444482,L,R,out,5,,8.352102e+00,-40.359012,52.611542,14,outcome_zone
3,2024,444482,L,R,out,6,,6.733365e+00,-12.514937,37.730677,12,outcome_zone
4,2024,444482,L,R,out,8,,1.431883e+07,-21.000016,43.567891,16,outcome_zone
...,...,...,...,...,...,...,...,...,...,...,...,...
6777,2024,808982,L,L,,<NA>,,2.095882e+00,-15.949253,38.343128,41,global
6778,2024,808982,L,R,out,5,,4.741955e+07,-7.000004,38.038760,10,outcome_zone
6779,2024,808982,L,R,out,<NA>,,-1.658179e+00,38.674106,42.363984,70,outcome
6780,2024,808982,L,R,single,<NA>,,-1.067791e+05,22.001115,29.510789,23,outcome
